In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

import nrrd
import numpy as np
import SimpleITK as sitk
import torch
from tqdm import tqdm

from utils import process_img_array, load_process_csv
from matplotlib import pyplot as plt
from experiments.utils import generate_seg
from experiments.model_utils import load_medsam

device = 'cuda' if torch.cuda.is_available() else 'cpu'; print(device)
medsam = load_medsam("../work_dir/MedSAM/medsam_vit_b.pth", device)

In [ ]:
data_folder = Path("/media/liushifeng/KINGSTON/ULS Jan 2025/ULS23/novel_data/ULS23_DeepLesion3D")
output_folder = Path("/home/liushifeng/Documents/GitHub/MedSAM/experiments/outputs/3dseg_with_bbox2d/ULS_DL3D_7Mar2025/")

In [ ]:
with open(data_folder / "train.txt") as f:
    train_names = [x.rstrip() for x in f.readlines()]
with open(data_folder / "val.txt") as f:
    val_names = [x.rstrip() for x in f.readlines()]

In [ ]:
# [Optional] load lesion type from DeepLesion
csv_folder = Path("/media/liushifeng/KINGSTON/DeepLesion Nov 2024/")
df = load_process_csv(csv_folder / "DL_info.csv")
LESION_TYPE_WINDOWS = {-1: None, 1: 'bone', 2: 'abdomen', 3: 'abdomen', 4: 'abdomen',
     5: 'lung', 6: 'abdomen', 7: 'abdomen', 8: 'abdomen'}
df['ct_window'] = df['Coarse_lesion_type'].map(LESION_TYPE_WINDOWS)

In [ ]:
# load a sample ct & seg
for filename in val_names[:1]:
    img_path = data_folder / "images" / filename
    seg_path = data_folder / "labels" / filename

ct = sitk.ReadImage(img_path)
seg = sitk.ReadImage(seg_path)
ct_array = sitk.GetArrayFromImage(ct)
seg_array = sitk.GetArrayFromImage(seg)

### Segment with windowing

In [ ]:
window_types = []
for filename in tqdm(train_names):

    orig_name = filename.split("_lesion_")[0]
    window = df[df['File_name'].str.contains(orig_name)].iloc[:1]['ct_window'].item()

    if window is None:
        continue
    window_types.append(window)

    # window = None
    img_path = data_folder / "images" / filename
    seg_path = data_folder / "labels" / filename

    print(f"Segmenting {filename} in {window=}")

    vol_seg = generate_seg(
        img_path, seg_path, medsam,
        None, # "ULS_full_10Feb2025_viz",
        window,
        plot=True
    )
    # save nrrd 3d seg for eval
    nrrd.write(f"{output_folder}/{filename}_pred.nrrd", vol_seg)

### Segment 2D slices with GT 2D bbox

In [ ]:
data_folder = Path("/media/liushifeng/KINGSTON/ULS Jan 2025/ULS23/novel_data/ULS23_DeepLesion3D")
output_folder = Path("/home/liushifeng/Documents/GitHub/MedSAM/experiments/outputs/3dseg_with_bbox2d/ULS_DL3D_7Mar2025/")

In [ ]:
from experiments.utils import generate_seg_on_slices

# Perform seg with windowing
res = []
window_types = []
for filename in tqdm(val_names):

    orig_name = filename.split("_lesion_")[0]
    window = df[df['File_name'].str.contains(orig_name)].iloc[:1]['ct_window'].item()

    if window is None:
        continue
    window_types.append(window)

    # window = None
    img_path = data_folder / "images" / filename
    seg_path = data_folder / "labels" / filename

    dices, vol_scores = generate_seg_on_slices(
        img_path, seg_path, medsam
    )
    for i, (d, v) in enumerate(zip(dices, vol_scores)):
        res.append(
            {
                "filename": filename,
                "lesion_num": i,
                "dice": d,
                "vs": v,
            }
        )

    # # save nrrd 3d seg for eval
    # nrrd.write(f"{output_folder}/{filename}_pred.nrrd", vol_seg)

In [ ]:
import pandas as pd
d = pd.DataFrame(res)

In [ ]:
d[['dice', 'vs']].describe()